In [1]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'mesolitica-storage.json'

In [3]:
from google.cloud import storage
client = storage.Client()
bucket = client.bucket('mesolitica-general')

In [4]:
best = '508000'
!mkdir t5-base-summary
model = best

mkdir: cannot create directory ‘t5-base-summary’: File exists


In [5]:
# blob = bucket.blob(f't5-base-summary/model.ckpt-{model}.data-00000-of-00002')
# blob.download_to_filename(f't5-base-summary/model.ckpt-{model}.data-00000-of-00002')

# blob = bucket.blob(f't5-base-summary/model.ckpt-{model}.data-00001-of-00002')
# blob.download_to_filename(f't5-base-summary/model.ckpt-{model}.data-00001-of-00002')

# blob = bucket.blob(f't5-base-summary/model.ckpt-{model}.index')
# blob.download_to_filename(f't5-base-summary/model.ckpt-{model}.index')

# blob = bucket.blob(f't5-base-summary/model.ckpt-{model}.meta')
# blob.download_to_filename(f't5-base-summary/model.ckpt-{model}.meta')

# blob = bucket.blob('t5-base-summary/checkpoint')
# blob.download_to_filename('t5-base-summary/checkpoint')

# blob = bucket.blob('t5-base-summary/operative_config.gin')
# blob.download_to_filename('t5-base-summary/operative_config.gin')

In [6]:
# with open('t5-base-summary/checkpoint', 'w') as fopen:
#     fopen.write(f'model_checkpoint_path: "model.ckpt-{model}"')

In [7]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
import t5

In [9]:
model = t5.models.MtfModel(
    model_dir='t5-base-summary',
    tpu=None,
    tpu_topology=None,
    model_parallelism=2,
    batch_size=5,
    sequence_length={"inputs": 1024, "targets": 1024},
    learning_rate_schedule=0.003,
    save_checkpoints_steps=5000,
    keep_checkpoint_max=3,
    iterations_per_loop=100,
    mesh_shape="model:1,batch:1", 
    mesh_devices=["gpu:0"]
)

In [10]:
!rm -rf output/*

In [11]:
saved_model_path = model.export(
    'output',
    checkpoint_step=-1,
    beam_size=1,
    temperature=0,
    sentencepiece_model_path='sp10m.cased.t5.model'
)

INFO:tensorflow:Using config: {'_model_dir': 't5-base-summary', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f05257c1978>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100, num_shards=None, num_cores_per_replica=1, p

In [12]:
saved_model_path.decode()

'output/1603121610'

In [14]:
import tensorflow_text  

tf.compat.v1.reset_default_graph()
sess = tf.InteractiveSession()
meta_graph_def = tf.compat.v1.saved_model.load(sess, ["serve"], 'output/1603121610')
signature_def = meta_graph_def.signature_def["serving_default"]
pred = lambda x: sess.run(
    fetches=signature_def.outputs["outputs"].name, 
    feed_dict={signature_def.inputs["input"].name: x}
)

/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


INFO:tensorflow:Restoring parameters from output/1603121610/variables/variables


In [15]:
string = """
Amanah Kedah berpendapat jika ada Adun Pakatan Harapan atau Bersatu negeri itu mahu berpaling tadah memberikan sokongan kepada kumpulan Muafakat Nasional, mereka perku membuat kenyataan rasmi mengenainya.

Pengerusi Amanah Kedah, Phahrolrazi Mohd Zawawi, berkata disebabkan tiada mana-mana Adun membuat kenyataan berhubung isu itu maka kerajaan negeri berpendapat tiada apa-apa yang berlaku.

Ditemui media selepas mengadakan pertemuan tertutup lebih sejam dengan Menteri Besar, Mukhriz Mahathir, hari ini Phahrolrazi berkata pihaknya juga mendapati kerajaan negeri masih berfungsi seperti biasa.

"Kami bincang keadaan semasa, ada juga kita sentuh (cubaan menukar kerajaan negeri), tetapi kita lihat kerajaan masih berfungsi.

"Tidak ada apa-apa kenyataan dari pihak sana (pembangkang) bahawa mereka sudah cukup majoriti setakat ini," katanya seperti dipetik BH Online.

Spekulasi mengenai pertukaran kerajaan menjadi kencang sejak semalam ekoran berlaku pertemuan tertutup pemimpin PAS dan Umno Kedah di Alor Setar semalam.

Turut hadir Setiausaha Agung PAS yang juga Menteri di Jabatan Perdana Menteri, Takiyuddin Hassan, dan Menteri Besar Terengganu, Dr Ahmad Samsuri Mokhtar.  

Cuba jatuhkan sejak dulu

Perkembangan itu berlaku kesan tindakan PKR memecat dan menggantung sejumlah anggota mereka baru-baru ini dan dipercayai memberi kesan terhadap pendirian wakil rakyat parti itu di Kedah.

Turut disebut-sebut akan beralih arah dalam perjalanan politik mereka ialah Adun Bersatu.

Untuk rekod berdasarkan pecahan parti PAS menguasai kerusi terbesar dalam DUN dan lazimnya pemimpin parti itu akan menjadi pilihan menjadi menteri besar jika berlaku pertukaran kerajaan.

Menurut Phahrolrazi, jika ada mana-mana wakil rakyat Bersatu atau PH mahu melompat, mereka wajar menyatakannya secara rasmi.

Tanpa kenyataan begitu, katanya, Amanah beranggapan isu perubahan kerajaan negeri masih bersifat spekulasi.

Timbalan Pengerusi Amanah Kedah, Dr Ismail Salleh, pula berkata ada kemungkinan Adun Bersatu, PH atau exco negeri tu yang sudah diumpan untuk membelakangkan mandat rakyat.

Beliau yang juga exco Kedah berkata memang sejak dulu lagi PAS cuba menjatuhkan kerajaan negeri dengan memujuk Adun PH serta Bersatu bertindak seperti rakan mereka di Perak, Johor dan Selangor.
"""

In [16]:
import re

# minimum cleaning, just simply to remove newlines.
def cleaning(string):
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

string = cleaning(string)

In [17]:
r = pred([f'{string}'] * 5)
r

array([b'- Kerajaan negeri di Kedah berpendapat bahawa jika ada yang mahu melompat, mereka harus membuat kenyataan rasmi mengenainya. Pengerusi Amanah Phahrolrazi Mohd Zawawi mengatakan bahawa jika ada yang mahu melompat, mereka harus menyatakannya secara rasmi, lapor Ap. "Kami juga melihat kerajaan masih berfungsi," katanya. "Tidak ada apa-apa kenyataan dari pihak sana (pembangkang) bahawa mereka sudah cukup majoriti setakat ini. " Spekulasi mengenai pertukaran kerajaan menjadi kencang sejak semalam, ketika pemimpin PAS dan Umno bertemu di Alor Setar.',
       b'- Kerajaan negeri di Kedah berpendapat bahawa jika ada yang mahu melompat, mereka harus membuat kenyataan rasmi mengenainya. Pengerusi Amanah Phahrolrazi Mohd Zawawi mengatakan bahawa jika ada yang mahu melompat, mereka harus menyatakannya secara rasmi, lapor Ap. "Kami juga melihat kerajaan masih berfungsi," katanya. "Tidak ada apa-apa kenyataan dari pihak sana (pembangkang) bahawa mereka sudah cukup majoriti setakat ini. " Sp

In [18]:
r = pred([f'ringkasan: {string}'] * 5)
r

array([b'Kerajaan negeri mengatakan tiada kenyataan dibuat mengenai isu. Spekulasi mengenai pertukaran kerajaan menjadi kencang sejak semalam. Ahli politik yang disebut-sebut akan beralih arah dalam perjalanan politik mereka adalah Adun Bersatu',
       b'Kerajaan negeri mengatakan tiada kenyataan dibuat mengenai isu. Spekulasi mengenai pertukaran kerajaan menjadi kencang sejak semalam. Ahli politik yang disebut-sebut akan beralih arah dalam perjalanan politik mereka adalah Adun Bersatu',
       b'Kerajaan negeri mengatakan tiada kenyataan dibuat mengenai isu. Spekulasi mengenai pertukaran kerajaan menjadi kencang sejak semalam. Ahli politik yang disebut-sebut akan beralih arah dalam perjalanan politik mereka adalah Adun Bersatu',
       b'Kerajaan negeri mengatakan tiada kenyataan dibuat mengenai isu. Spekulasi mengenai pertukaran kerajaan menjadi kencang sejak semalam. Ahli politik yang disebut-sebut akan beralih arah dalam perjalanan politik mereka adalah Adun Bersatu',
       b'Ker